<a href="https://colab.research.google.com/github/feknall/machinelearning-playground/blob/main/DeepLearning_Assignment_2_File_Multiple_Optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installation

In [ ]:
%load_ext tensorboard

In [ ]:
! pip install tensorflow --upgrade

In [ ]:
! pip install keras-tuner --upgrade

# Keras

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape((x_train.shape[0], -1)).astype('float32') / 255
x_test = x_test.reshape((x_test.shape[0], -1)).astype('float32') / 255

scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder()
one_hot_encoder.fit(y_train.reshape(-1, 1))
y_train_one_hot_encoded = one_hot_encoder.transform(y_train.reshape(-1, 1)).toarray()

In [ ]:
experiment_train_size = 60000
X = x_train[:experiment_train_size]
Y = y_train_one_hot_encoded[:experiment_train_size]

In [ ]:
X

In [ ]:
y_train_one_hot_encoded.shape

In [ ]:
y.shape

### Keras Adam

In [ ]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import tensorflow as tf

def plot_tensorflow_log(path):

    # Loading too much data is slow...
    tf_size_guidance = {
        # 'compressedHistograms': 100,
        # 'images': 0,
        # 'scalars': 10000,
        # 'histograms': 1000
        'tensors': 200
    }

    event_acc = EventAccumulator(path, tf_size_guidance)
    event_acc.Reload()

    print(event_acc.Tags())

    training_accuracies = event_acc.Tensors('epoch_accuracy')
    tensor_np = tf.make_ndarray(training_accuracies[-1].tensor_proto)

    # print(tensor_np)

    y = list()
    x = list()
    for i in range(len(training_accuracies)):
        y.append(tf.make_ndarray(training_accuracies[i].tensor_proto))
        x.append(i)

    return x, y

In [ ]:
log_file = "/content/drive/MyDrive/deep-learning/sgd/learning_rate_project/callback/0000/execution0/train/events.out.tfevents.1648605315.f2bd2e92bee4.73.1.v2"
x, y = plot_tensorflow_log(log_file)

## Optimizer

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Dense

def build_model(hp):
  optimizer = hp.Choice("optimizer", ["sgd", "rmsprop", "adam", "adadelta"])
  if optimizer == "sgd":
    optimizer = keras.optimizers.SGD(learning_rate=0.001)
  elif optimizer == "rmsprop":
    optimizer = keras.optimizers.RMSprop(learning_rate=0.001)
  elif optimizer == "adam":
    optimizer = keras.optimizers.Adam(learning_rate=0.001)
  elif optimizer == "adadelta":
    optimizer = keras.optimizers.Adadelta(learning_rate=0.001)

  model = keras.Sequential()
  model.add(Dense(10, kernel_initializer='uniform', input_shape=(784,), activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=["accuracy"])
  return model

In [ ]:
import keras_tuner as kt
from tensorflow import keras

tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=1000,
    executions_per_trial=1,
    overwrite=True,
    directory="/content/drive/MyDrive/deep-learning/optimizer/",
    project_name="optimizer_project"
)

tuner.search(X, 
             Y, 
             epochs=100, 
             validation_split=0.2, 
             callbacks=[tf.keras.callbacks.TensorBoard("/content/drive/MyDrive/deep-learning/optimizer/optimizer_project/callback")],
          
)

In [ ]:
! rm -rf /content/drive/MyDrive/deep-learning/optimizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/deep-learning/optimizer/optimizer_project/callback

In [ ]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt


log_file = "/content/drive/MyDrive/deep-learning/sgd/learning_rate_project/callback/0000/execution0/train/events.out.tfevents.1648605315.f2bd2e92bee4.73.1.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='0.1 learning rate - train')

log_file = "/content/drive/MyDrive/deep-learning/sgd/learning_rate_project/callback/0003/execution0/train/events.out.tfevents.1648605432.f2bd2e92bee4.73.10.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='0.01 learning rate - train')

log_file = "/content/drive/MyDrive/deep-learning/sgd/learning_rate_project/callback/0001/execution0/train/events.out.tfevents.1648605356.f2bd2e92bee4.73.4.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='0.001 learning rate - train')

log_file = "/content/drive/MyDrive/deep-learning/sgd/learning_rate_project/callback/0002/execution0/train/events.out.tfevents.1648605395.f2bd2e92bee4.73.7.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='0.0001 learning rate - train')


log_file = "/content/drive/MyDrive/deep-learning/sgd/learning_rate_project/callback/0000/execution0/validation/events.out.tfevents.1648605318.f2bd2e92bee4.73.2.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, '--', label='0.1 learning rate - validation')

log_file = "/content/drive/MyDrive/deep-learning/sgd/learning_rate_project/callback/0003/execution0/validation/events.out.tfevents.1648605434.f2bd2e92bee4.73.11.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, '--', label='0.01 learning rate - validation')

log_file = "/content/drive/MyDrive/deep-learning/sgd/learning_rate_project/callback/0001/execution0/validation/events.out.tfevents.1648605358.f2bd2e92bee4.73.5.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, '--', label='0.001 learning rate - validation')

log_file = "/content/drive/MyDrive/deep-learning/sgd/learning_rate_project/callback/0002/execution0/validation/events.out.tfevents.1648605396.f2bd2e92bee4.73.8.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, '--', label='0.0001 learning rate - validation')


plt.xlabel("Steps")
plt.ylabel("Accuracy")
plt.title("Training Progress - SGD")
plt.legend(loc='down right', frameon=True)
plt.show()

## Initializer

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras import initializers

def build_model(hp):
  initializer = hp.Choice("initializer", ['random_normal',
                                          'random_uniform', 
                                          'zeros', 
                                          'ones'])
  
  model = keras.Sequential()
  model.add(Dense(10, kernel_initializer=initializer, input_shape=(784,), activation='softmax'))

  opt = keras.optimizers.SGD(learning_rate=0.001)
  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
  return model

In [ ]:
import keras_tuner as kt
from tensorflow import keras

tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=1000,
    executions_per_trial=1,
    overwrite=True,
    directory="/content/drive/MyDrive/deep-learning/sgd/",
    project_name="initializer_project"
)

tuner.search(X, 
             Y, 
             epochs=20, 
             validation_split=0.2, 
             callbacks=[tf.keras.callbacks.TensorBoard("/content/drive/MyDrive/deep-learning/sgd/initializer_project/callback")],
          
)

In [ ]:
! rm -rf /content/drive/MyDrive/deep-learning/sgd/initializer_project/callback

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/deep-learning/sgd/initializer_project/callback

In [ ]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt



log_file = "/content/drive/MyDrive/deep-learning/sgd/initializer_project/callback/0000/execution0/train/events.out.tfevents.1648606551.f2bd2e92bee4.73.19.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='ones - train')

log_file = "/content/drive/MyDrive/deep-learning/sgd/initializer_project/callback/0003/execution0/train/events.out.tfevents.1648606667.f2bd2e92bee4.73.28.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='zeros - train')

log_file = "/content/drive/MyDrive/deep-learning/sgd/initializer_project/callback/0001/execution0/train/events.out.tfevents.1648606590.f2bd2e92bee4.73.22.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='random normal - train')

log_file = "/content/drive/MyDrive/deep-learning/sgd/initializer_project/callback/0002/execution0/train/events.out.tfevents.1648606628.f2bd2e92bee4.73.25.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='random uniform - train')


log_file = "/content/drive/MyDrive/deep-learning/sgd/initializer_project/callback/0000/execution0/validation/events.out.tfevents.1648606554.f2bd2e92bee4.73.20.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, '--', label='ones - validation')

log_file = "/content/drive/MyDrive/deep-learning/sgd/initializer_project/callback/0003/execution0/validation/events.out.tfevents.1648606669.f2bd2e92bee4.73.29.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, '--', label='zeros - validation')

log_file = "/content/drive/MyDrive/deep-learning/sgd/initializer_project/callback/0001/execution0/validation/events.out.tfevents.1648606592.f2bd2e92bee4.73.23.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, '--', label='random normal - validation')

log_file = "/content/drive/MyDrive/deep-learning/sgd/initializer_project/callback/0001/execution0/validation/events.out.tfevents.1648606592.f2bd2e92bee4.73.23.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, '--', label='random uniform - validation')

plt.xlabel("Steps")
plt.ylabel("Accuracy")
plt.title("Training Progress - SGD")
plt.legend(loc='down right', frameon=True)
plt.show()

## Batch Size

In [ ]:
class MyHyperModel(kt.HyperModel):
    def build(self, hp):  
      model = keras.Sequential()
      model.add(Dense(10, kernel_initializer='uniform', input_shape=(784,), activation='softmax'))

      opt = keras.optimizers.SGD(learning_rate=0.001)
      model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
      return model

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", [8, 32, 128, 512, 2048]),
            **kwargs,
        )

In [ ]:
import keras_tuner as kt
from tensorflow import keras

tuner = kt.RandomSearch(
    hypermodel=MyHyperModel(),
    objective="val_accuracy",
    max_trials=1000,
    executions_per_trial=1,
    overwrite=True,
    directory="/content/drive/MyDrive/deep-learning/sgd/",
    project_name="batch_size_project"
)

tuner.search(X, 
             Y, 
             epochs=20, 
             validation_split=0.2, 
             callbacks=[tf.keras.callbacks.TensorBoard("/content/drive/MyDrive/deep-learning/sgd/batch_size_project/callback")],
          
)

In [ ]:
! rm -rf /content/drive/MyDrive/deep-learning/sgd/batch_size_project/callback

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/deep-learning/sgd/batch_size_project/callback

In [ ]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt


log_file = "/content/drive/MyDrive/deep-learning/sgd/batch_size_project/callback/0003/execution0/train/events.out.tfevents.1648607170.f2bd2e92bee4.73.41.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='8 batch size - train')

log_file = "/content/drive/MyDrive/deep-learning/sgd/batch_size_project/callback/0002/execution0/train/events.out.tfevents.1648607134.f2bd2e92bee4.73.38.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='32 batch size - train')

log_file = "/content/drive/MyDrive/deep-learning/sgd/batch_size_project/callback/0005/execution0/train/events.out.tfevents.1648607312.f2bd2e92bee4.73.47.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='128 batch size - train')

log_file = "/content/drive/MyDrive/deep-learning/sgd/batch_size_project/callback/0004/execution0/train/events.out.tfevents.1648607306.f2bd2e92bee4.73.44.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='512 batch size - train')

log_file = "/content/drive/MyDrive/deep-learning/sgd/batch_size_project/callback/0001/execution0/train/events.out.tfevents.1648607130.f2bd2e92bee4.73.35.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='2048 batch size - train')



log_file = "/content/drive/MyDrive/deep-learning/sgd/batch_size_project/callback/0003/execution0/validation/events.out.tfevents.1648607176.f2bd2e92bee4.73.42.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, '--', label='8 batch size - validation')

log_file = "/content/drive/MyDrive/deep-learning/sgd/batch_size_project/callback/0002/execution0/validation/events.out.tfevents.1648607135.f2bd2e92bee4.73.39.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, '--', label='32 batch size - validation')

log_file = "/content/drive/MyDrive/deep-learning/sgd/batch_size_project/callback/0005/execution0/validation/events.out.tfevents.1648607313.f2bd2e92bee4.73.48.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, '--', label='128 batch size - validation')

log_file = "/content/drive/MyDrive/deep-learning/sgd/batch_size_project/callback/0004/execution0/validation/events.out.tfevents.1648607306.f2bd2e92bee4.73.45.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, '--', label='512 batch size - validation')

log_file = "/content/drive/MyDrive/deep-learning/sgd/batch_size_project/callback/0001/execution0/validation/events.out.tfevents.1648607130.f2bd2e92bee4.73.36.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, '--', label='2048 batch size - validation')


plt.xlabel("Steps")
plt.ylabel("Accuracy")
plt.title("Training Progress - SGD")
plt.legend(loc='down right', frameon=True)
plt.show()